In [ ]:
# solution by Sayan Chandra CS20M057
# CS6910-DEEP LEARNING- (prof.) Mitesh M. Khapra

In [1]:
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist, mnist
import numpy as np
from copy import deepcopy as clone


from sklearn.datasets import make_blobs
data, lables = make_blobs(n_samples=10000, centers=2, n_features=5, random_state=0)


# load dataset

In [2]:

def loadData():
    (trainX, trainy), (testX, testy) = fashion_mnist.load_data()
    N_inputs, dim_x_inp, dim_y_inp=len(trainX), len(trainX[0]), len(trainX[0][0])
    inputs=np.zeros((N_inputs, dim_x_inp*dim_y_inp))
    sum1=np.zeros((dim_x_inp*dim_y_inp))
    for i in range(N_inputs):
       inputs[i]=trainX[i].flatten()
       sum1+=inputs[i]
    mean1=sum1/N_inputs
    dim_inp=inputs.shape[1]

    sum2=np.zeros((dim_x_inp*dim_y_inp))
    testx=np.zeros((len(testX), dim_x_inp*dim_y_inp))
    for i in range(len(testX)):
      testx[i]=testX[i].flatten()
      sum2+=testx[i]
    mean2=sum2/len(testX)

    for i in range(N_inputs):
      inputs[i]-=mean1
      inputs[i]/=255
    trainx=inputs

    for i in range(len(testX)):
      testx[i]-=mean2
      testx[i]/=255
    return (trainx[6000: ], trainy[6000:], trainx[:6000], trainy[:6000], testx, testy)


In [3]:
trainx, trainy, valx, valy, testx, testy=loadData()
dim_inp=trainx.shape[1]

4423680/4422102 [==============================] - 0s 0us/step


In [5]:


## main variables inputs matrix, N_inputs, dim_inp

class MLSN (object): ## Multi layer Sigmoid Neurons
  def __init__(self, szInp, hiddenL, numOp, actvfunc, initfunc, lossfunc, lmda, vx, vy, xtest, ytest) :## size of inp, num of hidden layers, num of neurons per layer, num of output.
    self.szInp=szInp
    self.hiddenL=hiddenL
    self.numOp=numOp
    self.actvfunc=actvfunc
    self.initfunc=initfunc
    self.lossfunc=lossfunc
    self.lmda=lmda
    self.vx=vx
    self.vy=vy
    self.xtest=xtest
    self.ytest=ytest
    
    self.genericLayer=[szInp]+hiddenL+[numOp]
    print("Architechture of Neural Network (including i/p and o/p layer) ",self.genericLayer)
    

    
    self.weights = []

    if self.initfunc=="random":
      for i in range(len(self.genericLayer) - 1):
        w = np.random.random((self.genericLayer[i], self.genericLayer[i + 1]))
        (self.weights).append(w)
    elif self.initfunc=="xavier":
      for i in range(len(self.genericLayer) - 1):
        w = np.random.normal(0, 1/np.sqrt(self.genericLayer[i]), (self.genericLayer[i], self.genericLayer[i + 1]))
        (self.weights).append(w)

    

    self.biases=[]
    for i in range(len(self.genericLayer) - 1):
      b = np.random.normal(0, 1, self.genericLayer[i + 1])
      (self.biases).append(b)

    self.activations = []
    for i in range(len(self.genericLayer)):
      a = np.zeros(self.genericLayer[i])
      (self.activations).append(a)    

  def calculate_a_i(self, i, w):
    return np.dot(i,w)

  sigmoid = lambda self, pa: np.array([(np.exp(term) if term<0 else 1)/(1+np.exp(term if term<0 else -term)) for term in pa])
  tanh = lambda self, pa: (np.exp(pa-np.max(pa))-np.exp(-pa-np.max(pa)))/(np.exp(pa-np.max(pa))+np.exp(-pa-np.max(pa)))
  reLU = lambda self, pa: np.maximum(0,np.minimum(1e+200,pa))

  def calculate_h_i(self,pa):
    if self.actvfunc=="sigmoid" : return self.sigmoid(pa) 
    elif self.actvfunc=="tanh" :  return np.tanh(pa)
    elif self.actvfunc=="reLU" : return self.reLU(pa)+1e-8
    #return 1/(1+np.exp(-pa))

  def softmax(self,outpa):
    mx=np.max(outpa)
    outpa=outpa-mx
    return np.exp(outpa)/np.sum(np.exp(outpa)+1e-9)


  def feedFrwd(self,curinp,ws,bs):
    self.activations[0]=curinp
    for i in range(len(ws)-1):
      preActv=self.calculate_a_i(curinp, ws[i])+bs[i]
      actv=self.calculate_h_i(preActv)
      self.activations[i+1]=actv
      curinp=actv
    outPreActv=np.dot(curinp, ws[-1])+bs[-1]
    outActv=self.softmax(outPreActv)
    self.activations[-1]=outActv


  def justForwarding(self, xs):
      print("Weights and biases are initialised using "+self.initfunc+" and activation function is "+self.actvfunc+
            ".\nJust doing one time forward propagation(without back-propagation) for each data point.\nTaking first 20 data points.")
      for i in range(len(xs)):
        curinp=xs[i]
        self.feedFrwd(curinp,self.weights, self.biases)
        print("For point ",str(i+1)," Outputting a probability distribution over the 10 classes. ",self.activations[-1],".\nPredicted class ", np.argmax(self.activations[-1]))


## main(){}//
obj=MLSN(dim_inp, [16, 32], 10, "sigmoid", "xavier", "crossentropy", 0.0002, valx, valy, testx, testy)# activ func, initialztn,
obj.justForwarding(trainx[0:20])

Architechture of Neural Network (including i/p and o/p layer)  [784, 16, 32, 10]
Weights and biases are initialised using xavier and activation function is sigmoid.
Just doing one time forward propagation(without back-propagation) for each data point.

For point  1  Outputting a probability distribution over the 10 classes.  [0.05630292 0.074045   0.13077432 0.02065669 0.21122704 0.02762814
 0.13130758 0.06412068 0.13593174 0.14800587] .
Predicted class  4
For point  2  Outputting a probability distribution over the 10 classes.  [0.05653563 0.07442449 0.13061673 0.02081382 0.20840086 0.02761842
 0.12949204 0.06496514 0.13757779 0.14955507] .
Predicted class  4
For point  3  Outputting a probability distribution over the 10 classes.  [0.05714137 0.07327354 0.13042444 0.02119027 0.20692053 0.02732322
 0.13110683 0.06467064 0.13802592 0.14992324] .
Predicted class  4
For point  4  Outputting a probability distribution over the 10 classes.  [0.05701879 0.07389544 0.13034308 0.02107149 0.20